# Scale MotionDataset

In [1]:
%install-location /notebooks/language2motion.gt/swift-install
%install-swiftpm-flags -c release
%install '.package(path: "/notebooks/language2motion.gt")' Datasets ModelSupport

Installing packages:
	.package(path: "/notebooks/language2motion.gt")
		Datasets
		ModelSupport
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmp9cpcgi5q/swift-install
[1/2] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
Initializing Swift...
Installation complete!


In [2]:
import TensorFlow
import PythonKit
import Foundation
import Datasets
let np  = Python.import("numpy")

# load dataset

In [3]:
let dataURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/")
// let motionDatasetURL = dataURL.appendingPathComponent("motion_dataset.motion_flag.downsampled.sampled.490.plist")
let motionDatasetURL = dataURL.appendingPathComponent("motion_dataset.motion_flag.downsampled.39102.plist")
let motionDataset = MotionDataset(from: motionDatasetURL)

In [4]:
motionDataset.motionSamples.count

39102


In [5]:
let motions = motionDataset.motionSamples.map { Tensor($0.motionFramesArray) }
motions.count

39102


In [6]:
let motionsTensor = Tensor(concatenating: motions, alongAxis: 0)

In [7]:
motionsTensor.shape

▿ [3859087, 48]
  ▿ dimensions : 2 elements
    - 0 : 3859087
    - 1 : 48


# fit

In [8]:
var scaler = Scaler(X: motionsTensor)

In [15]:
scaler.mean

▿ Optional<Tensor<Float>>
  - some : [ -0.38311696,  -0.05046073, -0.014832868,     0.253187,  -0.04258638,  -0.05768465,
  -0.33914182,  0.014154105,   0.06511231,    0.4063905,     0.046295, -0.028090745,
   0.12675667, -0.054657895,  -0.16116373,     0.835924,  -0.08843774,   0.36008993,
   0.13677886,   0.30643317,    -0.550201,   0.09726817,   0.42231324,   -0.2188532,
  -0.04135671,  -0.17972438,  0.021227267,  -0.11209103,   0.11487589,  0.060222104,
  -0.06865891,    0.8407422,   0.22554651,    0.3077817,  -0.10211985,  -0.16363145,
   -0.4850693,   0.17845915,  -0.43086854,   0.15707436,  -0.06813503,   0.17313054,
  0.043280035,   0.07448136,  -0.09551967, 0.0029034032,   0.06761817,          1.0]


In [16]:
scaler.scale

▿ Optional<Tensor<Float>>
  - some : [0.25630662, 0.19122753, 0.20436308,  0.2841897,   0.218454, 0.37832755, 0.28846982, 0.21309112,
  0.3688422, 0.16146001, 0.17431882,  0.2346692, 0.22838809,  0.2633178, 0.23261021, 0.55567735,
  0.4687475, 0.35539836, 0.17040752,  0.2671834, 0.41134602, 0.57042116, 0.49726322,  0.4577841,
 0.24958144, 0.33135676,  0.7231112, 0.18645777, 0.23776391, 0.28606012, 0.25044712,  0.5806572,
 0.49198854, 0.37318504, 0.16607577, 0.27255774, 0.45386443, 0.67675894,  0.5414401, 0.46907118,
 0.28322077, 0.34545648, 0.71956897, 0.21011858, 0.32497472, 0.26462087,  1.7417394,        0.0]


# transform

In [9]:
let scaledMotions = motions.map {scaler.transform($0)}
scaledMotions.count

39102


In [10]:
let scaledMotionsTensor = Tensor(concatenating: scaledMotions, alongAxis: 0)
scaledMotionsTensor.shape

▿ [3859087, 48]
  ▿ dimensions : 2 elements
    - 0 : 3859087
    - 1 : 48


In [11]:
motionsTensor.min(alongAxes: 0)

[[-0.785398, -0.349066, -0.261799, -0.872665, -0.349066, -0.785398, -0.610865, -0.349066,
  -0.628319, -0.349066, -0.261799, -0.349066, -0.698132, -0.349066, -0.523599,       0.0,
    -1.5708, -0.872665, -0.349066, -0.785398,  -2.26893,  -1.22173,       0.0,  -1.22173,
  -0.523599,  -1.22173, -0.785398, -0.261799, -0.698132, -0.349066, -0.523599,       0.0,
    -1.5708, -0.872665,  -1.13446, -0.785398,  -2.26893,  -1.22173,  -2.79253,   -1.0472,
  -0.523599,   -1.5708, -0.785398, -0.261799,  -3.14158,  -3.14158,  -3.14159,       1.0]]


In [12]:
motionsTensor.max(alongAxes: 0)

[[0.261799, 0.349066, 0.261799, 0.610865, 0.349066, 0.785398, 0.471239, 0.349066, 0.628319, 0.523599,
  0.261799, 0.349066, 0.523599, 0.349066, 0.523599,  2.79253,   1.5708,  1.65806,  1.13446, 0.785398,
      -0.0,  3.31613,  2.79253,   1.0472, 0.349066,   1.5708,   1.5708, 0.261799, 0.523599, 0.349066,
  0.523599,  2.79253,   1.5708,  1.65806, 0.349066, 0.785398,     -0.0,  3.31613,     -0.0,  1.22173,
  0.349066,  1.22173,   1.5708, 0.261799,  3.14147,  3.14159,  3.14159,      1.0]]


In [13]:
scaledMotionsTensor.min(alongAxes: 0)

[[ -1.5695305,  -1.5615182,  -1.2084675,  -3.9616213,   -1.402948,  -1.9235008,  -0.9419466,
   -1.7045295,  -1.8800216,  -4.6789074,  -1.7674167,  -1.3677775,   -3.611785,  -1.1180713,
   -1.5581229,  -1.5043335,  -3.1623895,  -3.4686568,  -2.8510764,  -4.0864487,  -4.1783047,
   -2.3123233, -0.84927505,    -2.19072,  -1.9322042,  -3.1446638,  -1.1154927,  -0.8029055,
   -3.4193914,  -1.4307765,  -1.8165116,  -1.4479148,  -3.6511958,   -3.163167,  -6.2160797,
   -2.2812288,  -3.9303823,  -2.0689628,  -4.3618145,  -2.5673594,   -1.608159,  -5.0481915,
   -1.1516311,  -1.6004314,   -9.373222,  -11.882977,  -1.8425307,         inf]]


In [14]:
scaledMotionsTensor.max(alongAxes: 0)

[[  2.516189,  2.0892742,  1.3536294,  1.2585889,  1.7928368,  2.2284465,  2.8092394,  1.5716839,
   1.5269585,  0.7259292,  1.2362635,  1.6071846,  1.7375791,  1.5332192,  2.9438207,  3.5211189,
    3.539726,  3.6521554,   5.854678,  1.7926445,  1.3375624,  5.6429567,  4.7665234,  2.7656121,
   1.5643098,   5.282899,  2.1429245,  2.0052264,  1.7190293,  1.0097313,  2.3648021,   3.361343,
   2.7343187,   3.618254,  2.7167468,   3.481939,  1.0687537,  4.6363196, 0.79578245,  2.2697103,
   1.4730594,  3.0354025,  2.1228263,  0.8914854,   9.960743,   11.86107,  1.7648863,       -inf]]


In [17]:
scaledMotionsTensor.mean(alongAxes: 0)

[[  0.00058566907,  -0.00014246305,   0.00018626645,   -0.0026798095, -0.000120324534,
   -8.8962086e-05,     0.010811062,  -0.00010074143,   0.00056139944,     0.053243835,
    -0.0014333109,  -2.5688058e-05,   -0.0017064841,   0.00022383028,   0.00012128679,
    -0.0009607348,   0.00029105032,   0.00015001264,  -0.00019082395,   -0.0008759864,
    -0.0015709823,  -4.8914085e-06,   0.00062930275,    0.0002929995,    0.0030970366,
    0.00045534808,   -0.0010851173,    -0.010019801,   -0.0016683501,   -0.0025069315,
     0.0002528769,    0.0012997193,  -0.00023552909,   1.5439691e-05,    2.276903e-05,
    0.00012710787,   -0.0011508015,   0.00017871174,   -0.0002233062,   0.00029251663,
     0.0018250246,   -0.0005011925,   -5.658647e-05,    -0.008063436,    5.849401e-06,
    2.6024534e-06,  -7.4980875e-07,            -nan]]


In [18]:
scaledMotionsTensor.standardDeviation(alongAxes: 0)

[[ 1.0009422,  1.0039941, 0.99432963,  0.9993729,  1.0010659,  1.0002971,  1.0036199,  1.0069597,
   1.0010172,  0.9936275, 0.99428093,  0.9925947,  0.9990627,  0.9977377,   1.001611,  1.0000176,
   0.9998914,  0.9997027,  1.0002328,  1.0002052,  1.0000582, 0.99974597, 0.99996436,  1.0000213,
   0.9983597,  1.0003358,  1.0033832,  1.0161266,  1.0005475, 0.99592924,    1.00033,  0.9986916,
   0.9994303,  1.0001129,  1.0003976,  1.0002819,  1.0005496,  0.9996827,  1.0018895,  1.0013044,
  0.99571306,  1.0005453,  1.0027347,  1.0075767,  0.9995989,  0.9998769,  1.0003543,       -nan]]


In [22]:
let descaledMotions = scaledMotions.map {scaler.inverse_transform($0)}
let descaledMotionsTensor = Tensor(concatenating: descaledMotions, alongAxis: 0)
descaledMotionsTensor.shape

▿ [3859087, 48]
  ▿ dimensions : 2 elements
    - 0 : 3859087
    - 1 : 48


In [23]:
descaledMotionsTensor.min(alongAxes: 0)

[[  -0.785398,   -0.349066,   -0.261799,   -0.872665,   -0.349066,   -0.785398,   -0.610865,
    -0.349066,   -0.628319, -0.34906596,   -0.261799,   -0.349066,   -0.698132,   -0.349066,
    -0.523599,         0.0,     -1.5708, -0.87266505,   -0.349066,   -0.785398,    -2.26893,
   -1.2217299,         0.0,    -1.22173,   -0.523599,    -1.22173, -0.78539807,   -0.261799,
    -0.698132, -0.34906602,   -0.523599,         0.0,     -1.5708,  -0.8726649,    -1.13446,
    -0.785398,    -2.26893,  -1.2217299,    -2.79253,     -1.0472,   -0.523599,     -1.5708,
    -0.785398,   -0.261799,    -3.14158,    -3.14158,    -3.14159,         inf]]


In [25]:
descaledMotionsTensor.min(alongAxes: 0)-motionsTensor.min(alongAxes: 0)

[[           0.0,            0.0,            0.0,            0.0,            0.0,
             0.0,            0.0,            0.0,            0.0,  2.9802322e-08,
             0.0,            0.0,            0.0,            0.0,            0.0,
             0.0,            0.0, -5.9604645e-08,            0.0,            0.0,
             0.0,  1.1920929e-07,            0.0,            0.0,            0.0,
             0.0, -5.9604645e-08,            0.0,            0.0, -2.9802322e-08,
             0.0,            0.0,            0.0,  5.9604645e-08,            0.0,
             0.0,            0.0,  1.1920929e-07,            0.0,            0.0,
             0.0,            0.0,            0.0,            0.0,            0.0,
             0.0,            0.0,            inf]]


In [24]:
descaledMotionsTensor.max(alongAxes: 0)

[[0.26179898,   0.349066,   0.261799,   0.610865,   0.349066,   0.785398, 0.47123894,   0.349066,
    0.628319,   0.523599, 0.26179898,   0.349066,   0.523599,   0.349066,   0.523599,    2.79253,
      1.5708,    1.65806,  1.1344601,   0.785398,        0.0,    3.31613,    2.79253,     1.0472,
    0.349066,     1.5708,  1.5708001,   0.261799,   0.523599, 0.34906596,   0.523599,    2.79253,
      1.5708,    1.65806, 0.34906596,   0.785398,        0.0,    3.31613,        0.0,    1.22173,
    0.349066,    1.22173,     1.5708,   0.261799,  3.1414697,    3.14159,    3.14159,       -inf]]


In [26]:
descaledMotionsTensor.max(alongAxes: 0)-motionsTensor.max(alongAxes: 0)

[[-2.9802322e-08,            0.0,            0.0,            0.0,            0.0,
             0.0, -5.9604645e-08,            0.0,            0.0,            0.0,
  -2.9802322e-08,            0.0,            0.0,            0.0,            0.0,
             0.0,            0.0,            0.0,  1.1920929e-07,            0.0,
             0.0,            0.0,            0.0,            0.0,            0.0,
             0.0,  1.1920929e-07,            0.0,            0.0, -2.9802322e-08,
             0.0,            0.0,            0.0,            0.0, -2.9802322e-08,
             0.0,            0.0,            0.0,            0.0,            0.0,
             0.0,            0.0,            0.0,            0.0, -2.3841858e-07,
             0.0,            0.0,           -inf]]


# Check frame rate

In [27]:
motionDataset.motionSamples[0].timestepsArray

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5,
 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7]


In [28]:
motionDataset.motionSamples[0].timestepsArray.count

38


In [79]:
let lastTimes = motionDataset.motionSamples.map { (
    last: $0.timestepsArray.last!.scalar!, 
    count: $0.timestepsArray.count, 
    freq: Float($0.timestepsArray.count)/$0.timestepsArray.last!.scalar!
)}

In [85]:
lastTimes.filter { $0.freq > 14 && $0.count > 2 }

▿ 3 elements
  ▿ 0 : 3 elements
    - last : 0.333333
    - count : 5
    - freq : 15.000015
  ▿ 1 : 3 elements
    - last : 0.341667
    - count : 5
    - freq : 14.634132
  ▿ 2 : 3 elements
    - last : 0.35
    - count : 5
    - freq : 14.285714


In [60]:
(lastTimes.map { $0.count }).max()

▿ Optional<Int>
  - some : 500


In [61]:
(lastTimes.map { $0.last }).max()

▿ Optional<Float>
  - some : 71.3333


In [64]:
lastTimes[3000]

▿ 2 elements
  - last : 4.2
  - count : 43


In [65]:
let lastTimeCounts = lastTimes.map { $0.count }

In [86]:
lastTimeCounts[0..<10]

▿ 10 elements
  - 0 : 38
  - 1 : 38
  - 2 : 38
  - 3 : 38
  - 4 : 38
  - 5 : 38
  - 6 : 38
  - 7 : 38
  - 8 : 37
  - 9 : 37


120 Hz

In [67]:
lastTimes[38545]

▿ 2 elements
  - last : 40.025
  - count : 481


In [68]:
481/40.025

12.017489069331669


60 Hz

In [70]:
500/71.3333

7.009349069789286


In [72]:
let freqs = lastTimes.map { Float($0.count)/$0.last }

In [73]:
freqs[0..<10]

▿ 10 elements
  - 0 : 10.27027
  - 1 : 10.242587
  - 2 : 10.215054
  - 3 : 10.187668
  - 4 : 10.160428
  - 5 : 10.133333
  - 6 : 10.106383
  - 7 : 10.079576
  - 8 : 10.054348
  - 9 : 10.027101


In [74]:
freqs.min()

▿ Optional<Float>
  - some : 6.001399


In [75]:
freqs.max()

▿ Optional<Float>
  - some : inf


In [87]:
// freqs.sorted()